In [1]:
import nest_asyncio
nest_asyncio.apply()

from dataclasses import dataclass
import asyncio
from agents.extensions.visualization import draw_graph
import os 
from openai import AsyncOpenAI 
from dotenv import load_dotenv  
load_dotenv() 
from agents import (
    Agent,
    Runner, 
    set_tracing_disabled,OpenAIChatCompletionsModel,enable_verbose_stdout_logging
)   
 

# enable_verbose_stdout_logging()
set_tracing_disabled(disabled=True)
API_KEY=os.getenv("GEMINI_API_KEY")

if  not API_KEY  :
    raise ValueError("Please set EXAMPLE_BASE_URL, EXAMPLE_API_KEY, EXAMPLE_MODEL_NAME via env var or code.")

 
client = AsyncOpenAI(base_url="https://generativelanguage.googleapis.com/v1beta/openai/",api_key=API_KEY,)

model = OpenAIChatCompletionsModel(model="gemini-2.0-flash",openai_client=client)

In [6]:
import asyncio
from agents import Agent, run_demo_loop

async def main() -> None:
    agent = Agent(name="Assistant", instructions="You are a helpful assistant.",model=model)
    await run_demo_loop(agent)

if __name__ == "__main__":
    asyncio.run(main())

ImportError: cannot import name 'run_demo_loop' from 'agents' (c:\Users\arman\AppData\Local\Programs\Python\Python313\Lib\site-packages\agents\__init__.py)

In [8]:
!pip install -U openai-agents

Defaulting to user installation because normal site-packages is not writeable
  Attempting uninstall: openai-agents
    Found existing installation: openai-agents 0.0.18
    Uninstalling openai-agents-0.0.18:
      Successfully uninstalled openai-agents-0.0.18


In [ ]:
import asyncio
from typing import Any, Awaitable
from agents import Handoff,Agent
from dataclasses import dataclass
 
# Disable tracing to reduce log noise
set_tracing_disabled(True)
 
# Define a dataclass for structured output
@dataclass
class HistoryResponse:
    summary: str
    period: str | None

# Define JSON schema for handoff input (period is required to trigger potential error)
history_schema = {
    "type": "object",
    "properties": {
        "period": {
            "type": "string",
            "description": "The historical period to query (e.g., 'Partition of India', 'Mughal era')"
        }
    },
    "required": ["period"]  # Strict requirement
}

# Define specialized agent
pak_agent = Agent(
    name="pak_agent",
    instructions="You have knowledge of Pakistan history. For broad queries like 'Pakistan history', provide a brief overview of key events, such as the Indus Valley Civilization, the Mughal era, the Pakistan Movement, and the Partition of India, and return a JSON object with 'summary' and 'period' set to null. For specific queries, provide detailed information and include the period. Return as a JSON object with 'summary' and 'period' fields.",
    model=model,
    handoff_description="Handles queries related to Pakistan history.",
    output_type=HistoryResponse
)

# Define async function for on_invoke_handoff
async def invoke_pak_handoff(ctx, input_json: str | None) -> Agent:
    print(f"Handoff to pak_agent triggered with input: {input_json}")
    if input_json:
        import json
        try:
            input_data = json.loads(input_json)
            print(f"Received period for Pakistan history: {input_data.get('period', 'None')}")
        except json.JSONDecodeError:
            raise ValueError("Invalid JSON input received")
    await asyncio.sleep(0)
    return pak_agent

# Create Handoff object
pak_handoff = Handoff(
    tool_name="transfer_to_pak_agent",
    tool_description="Transfers the query to an agent specialized in Pakistan history. Requires a historical period.",
    input_json_schema=history_schema,
    on_invoke_handoff=invoke_pak_handoff,
    agent_name=pak_agent.name,
    strict_json_schema=True
)

# Create the triage agent
triage_agent = Agent(
    name="triage_agent",
    instructions="Determine which agent to use based on the user's question. If the query mentions 'Pakistan history', use the transfer_to_pak_agent tool with a 'period' argument (e.g., {'period': 'Partition of India'}). If no specific period is mentioned, respond with 'I don't know'.",
    model=model,
    handoffs=[pak_handoff]
)

# Main function to run the agent and capture errors
async def main(user_input: str) -> str:
    print(f"Processing input: {user_input}")
    try:
        resp = await Runner.run(starting_agent=triage_agent, input=user_input)
        
        print("Runner completed")
        if isinstance(resp.final_output, HistoryResponse):
            return f"Summary: {resp.final_output.summary}\nPeriod: {resp.final_output.period or 'Not specified'}"
        return str(resp.final_output)
    except Exception as e:
        if isinstance(e, Exception) and hasattr(e, "run_error_details"):
            error_details: RunErrorDetails = e.run_error_details
            print(f"Error occurred: {str(error_details)}")
            return f"Error: {str(error_details)}"
        return f"Unexpected error: {str(e)}"

# Test the setup
if __name__ == "__main__":
    test_queries = [
        "what is pakistan history",
        "Tell me about the Partition of India in Pakistan history"
    ]
    for query in test_queries:
        print(f"\nQuery: {query}")
        final_resp = asyncio.run(main(query))
        print(f"Response: {final_resp}")


Query: what is pakistan history
Processing input: what is pakistan history
Runner completed
Response: I don't know


Query: Tell me about the Partition of India in Pakistan history
Processing input: Tell me about the Partition of India in Pakistan history
Handoff to pak_agent triggered with input: {"period":"Partition of India"}
Received period for Pakistan history: Partition of India
Runner completed
Response: Summary: The Partition of India in 1947 was a pivotal event in Pakistan's history, marking its emergence as an independent nation. It led to the division of British India into two separate states: India and Pakistan. This partition resulted in widespread communal violence, displacement, and immense human suffering as millions of people were forced to migrate across the newly created borders. The events surrounding the Partition continue to shape the socio-political landscape of both Pakistan and India.
Period: 1947


In [3]:
import asyncio
from openai.types.responses import ResponseTextDeltaEvent
from agents import Agent, Runner

async def main():
    agent = Agent(
        name="Joker",
        instructions="You are a helpful assistant.",
        model=model
    )

    result = Runner.run_streamed(agent, input="Please tell me 5 jokes.")
    async for event in result.stream_events():
        if event.type == "raw_response_event" and isinstance(event.data, ResponseTextDeltaEvent):
            print(event.data.delta, end="", flush=True)


if __name__ == "__main__":
    asyncio.run(main())

Okay, here are 5 jokes for you:

1.  Why don't scientists trust atoms? Because they make up everything!
2.  Parallel lines have so much in common. It's a shame they'll never meet.
3.  Why did the scarecrow win an award? Because he was outstanding in his field!
4.  What do you call a lazy kangaroo? Pouch potato!
5.  Why did the bicycle fall over? Because it was two tired!


In [5]:
import asyncio
from openai.types.responses import ResponseTextDeltaEvent
from agents import Agent, Runner

async def main():
    agent = Agent(
        name="Joker",
        instructions="You are a helpful assistant.",
        model=model
    )

    result = Runner.run_streamed(agent, input="Please tell me 5 jokes.")
    async for event in result.stream_events():
        # We'll ignore the raw responses event deltas
        if event.type == "raw_response_event":
            continue
        # When the agent updates, print that
        elif event.type == "agent_updated_stream_event":
            print(f"Agent updated: {event.new_agent.name}")
            continue
        # When items are generated, print them
        elif event.type == "run_item_stream_event":
            if event.item.type == "tool_call_item":
                print("-- Tool was called")
            elif event.item.type == "tool_call_output_item":
                print(f"-- Tool output: {event.item.output}")
            elif event.item.type == "message_output_item":
                print(f"-- Message output:\n {ItemHelpers.text_message_output(event.item)}")
            else:
                pass  # Ignore other event types

    print("=== Run complete ===")

if __name__ == "__main__":
    asyncio.run(main())

Agent updated: Joker
-- Message output:
 Alright, here are 5 jokes for you:

1.  Why don't scientists trust atoms?
    Because they make up everything!

2.  What do you call a lazy kangaroo?
    Pouch potato!

3.  Why did the bicycle fall over?
    Because it was two tired!

4.  Parallel lines have so much in common.
    It's a shame they'll never meet.

5.  Why did the scarecrow win an award?
    Because he was outstanding in his field!

=== Run complete ===


In [4]:
import asyncio
import random
from agents import Agent, ItemHelpers, Runner, function_tool

@function_tool
def how_many_jokes() -> int:
    return random.randint(1, 10)


async def main():
    agent = Agent(
        name="Joker",
        instructions="First call the `how_many_jokes` tool, then tell that many jokes.",
        tools=[how_many_jokes],
        model=model
    )

    result = Runner.run_streamed(
        agent,
        input="Hello",
        
    )
    print("=== Run starting ===")

    async for event in result.stream_events():
        # We'll ignore the raw responses event deltas
        if event.type == "raw_response_event":
            continue
        # When the agent updates, print that
        elif event.type == "agent_updated_stream_event":
            print(f"Agent updated: {event.new_agent.name}")
            continue
        # When items are generated, print them
        elif event.type == "run_item_stream_event":
            if event.item.type == "tool_call_item":
                print("-- Tool was called")
            elif event.item.type == "tool_call_output_item":
                print(f"-- Tool output: {event.item.output}")
            elif event.item.type == "message_output_item":
                print(f"-- Message output:\n {ItemHelpers.text_message_output(event.item)}")
            else:
                pass  # Ignore other event types

    print("=== Run complete ===")


if __name__ == "__main__":
    asyncio.run(main())

=== Run starting ===
Agent updated: Joker
-- Tool was called
-- Tool output: 4
-- Message output:
 I am programmed to tell 4 jokes.

Why don't scientists trust atoms?

Because they make up everything!

What do you call a lazy kangaroo?

Pouch potato!

Why did the scarecrow win an award?

Because he was outstanding in his field!

Why did the bicycle fall over?

Because it was two tired!

=== Run complete ===
